In [83]:
import QuantLib as ql

# Set up the yield term structure (5% flat forward rate)
crv = ql.FlatForward(2, ql.TARGET(), 0.05, ql.Actual360())  # Flat forward curve at 5%
usd3mcurve = ql.YieldTermStructureHandle(crv)                      # Wrap curve in a handle

model=ql.G2(usd3mcurve)
optimization_method = ql.LevenbergMarquardt(1e-8, 1e-8, 1e-8)
end_criteria = ql.EndCriteria(1000, 500, 1e-8, 1e-8, 1e-8)
periods = [ql.Period('1y'),ql.Period('2y'),ql.Period('5y'),ql.Period('7y'),ql.Period('10Y'),ql.Period('15Y')]
quotes = [ql.QuoteHandle(ql.SimpleQuote(0.0085)),ql.QuoteHandle(ql.SimpleQuote(0.0055)),ql.QuoteHandle(ql.SimpleQuote(0.0045)),
          ql.QuoteHandle(ql.SimpleQuote(0.0043)),ql.QuoteHandle(ql.SimpleQuote(0.0035)),ql.QuoteHandle(ql.SimpleQuote(0.0025))]
yts = usd3mcurve
index = ql.USDLibor(ql.Period('3m'),usd3mcurve)

helpers = [ql.CapHelper(i, j, index, ql.Quarterly, ql.Actual360(), False, usd3mcurve,0,ql.Normal) for i,j in zip(periods,quotes)]

for h in helpers:
    h.setPricingEngine(ql.TreeCapFloorEngine(model,20))

model.calibrate(helpers,optimization_method,end_criteria)
print(model.params())

model_vols=[]
for h in helpers:
    model_vols.append(h.impliedVolatility(h.modelValue(),1e-5,50,0,4))
print(model_vols)

[ 1.13806; 0.0139089; 1.13806; 0.0139089; -0.638518 ]
[0.007727663084877359, 0.006076001866294086, 0.004378030404544783, 0.0038199695731712647, 0.0033225695987325436, 0.002827470038621719]


In [84]:
# Define bond schedule
issue_date = ql.Date(15, 9, 2023)
maturity_date = ql.Date(15, 9, 2033)
tenor = ql.Period(ql.Semiannual)
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
settlement_days = 2
schedule = ql.Schedule(issue_date, maturity_date, tenor, calendar, ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Forward, False)

# Define bond coupon rates and redemption
coupon_rate = 0.05  # 5% coupon rate
coupons = [coupon_rate]
face_amount = 100.0
redemption = 100.0

# 5. Set up callable feature (callability schedule)
call_dates = [ql.Date(15, 9, 2025), ql.Date(15, 9, 2027), ql.Date(15, 9, 2029)]
call_prices = [100.0, 100.0, 100.0]  # Callable at different prices on different dates

callability_schedule = ql.CallabilitySchedule()
for call_date, call_price in zip(call_dates, call_prices):
    callability_price  = ql.BondPrice(call_price, ql.BondPrice.Clean)
    callability_schedule.append(
        ql.Callability(
            callability_price,    # Call price
            ql.Callability.Call,  # Call feature
            call_date      # Call date
        )
    )

# 6. Create the callable bond
callable_bond = ql.CallableFixedRateBond(
    settlement_days,  # Settlement days
    face_amount,      # Face value
    schedule,         # Bond schedule
    coupons,          # Coupon payments
    ql.ActualActual(ql.ActualActual.Bond), # Day count convention
    ql.Following,     # Business day convention
    redemption,       # Redemption value
    issue_date,       # Issue date
    callability_schedule
)

# 7. Set up the pricing engine for the callable bond using the calibrated G2++ model
callable_bond_engine = ql.TreeCallableFixedRateBondEngine(model, 100)  # Using 100 time steps
callable_bond.setPricingEngine(callable_bond_engine)

# 8. Price the callable bond
callable_bond_price = callable_bond.cleanPrice()
print("Callable Bond Price: ", callable_bond_price)

Callable Bond Price:  98.89007452991989


In [85]:
# Set up the yield term structure (5% flat forward rate)
crv = ql.FlatForward(2, ql.TARGET(), 0.05, ql.Actual360())  # Flat forward curve at 5%
usd3mcurve = ql.YieldTermStructureHandle(crv)                      # Wrap curve in a handle

model=ql.HullWhite(usd3mcurve)
optimization_method = ql.LevenbergMarquardt(1e-8, 1e-8, 1e-8)
end_criteria = ql.EndCriteria(1000, 500, 1e-8, 1e-8, 1e-8)
periods = [ql.Period('1y'),ql.Period('2y'),ql.Period('5y'),ql.Period('7y'),ql.Period('10Y'),ql.Period('15Y')]
quotes = [ql.QuoteHandle(ql.SimpleQuote(0.0085)),ql.QuoteHandle(ql.SimpleQuote(0.0055)),ql.QuoteHandle(ql.SimpleQuote(0.0045)),
          ql.QuoteHandle(ql.SimpleQuote(0.0043)),ql.QuoteHandle(ql.SimpleQuote(0.0035)),ql.QuoteHandle(ql.SimpleQuote(0.0025))]
index = ql.USDLibor(ql.Period('3m'),usd3mcurve)
helpers = [ql.CapHelper(i, j, index, ql.Quarterly, ql.Actual360(), False, usd3mcurve,0,ql.Normal) for i,j in zip(periods,quotes)]
for h in helpers:
    h.setPricingEngine(ql.TreeCapFloorEngine(model,20))
model.calibrate(helpers,optimization_method,end_criteria)
print(model.params())

model_vols=[]
for h in helpers:
    model_vols.append(h.impliedVolatility(h.modelValue(),1e-5,50,0,4))
print(model_vols)

[ 3.87276; 0.0237309 ]
[0.007917292052856541, 0.0058198729789102205, 0.004536374051231691, 0.003897472507141789, 0.00332474117280105, 0.0027983757851586443]


In [86]:
# Define bond schedule
issue_date = ql.Date(15, 9, 2023)
maturity_date = ql.Date(15, 9, 2033)
tenor = ql.Period(ql.Semiannual)
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
settlement_days = 2
schedule = ql.Schedule(issue_date, maturity_date, tenor, calendar, ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Forward, False)

# Define bond coupon rates and redemption
coupon_rate = 0.05  # 5% coupon rate
coupons = [coupon_rate]
face_amount = 100.0
redemption = 100.0

# 5. Set up callable feature (callability schedule)
call_dates = [ql.Date(15, 9, 2025), ql.Date(15, 9, 2027), ql.Date(15, 9, 2029)]
call_prices = [100.0, 100.0, 100.0]  # Callable at different prices on different dates

callability_schedule = ql.CallabilitySchedule()
for call_date, call_price in zip(call_dates, call_prices):
    callability_price  = ql.BondPrice(call_price, ql.BondPrice.Clean)
    callability_schedule.append(
        ql.Callability(
            callability_price,    # Call price
            ql.Callability.Call,  # Call feature
            call_date      # Call date
        )
    )

# 6. Create the callable bond
callable_bond = ql.CallableFixedRateBond(
    settlement_days,  # Settlement days
    face_amount,      # Face value
    schedule,         # Bond schedule
    coupons,          # Coupon payments
    ql.ActualActual(ql.ActualActual.Bond), # Day count convention
    ql.Following,     # Business day convention
    redemption,       # Redemption value
    issue_date,       # Issue date
    callability_schedule
)

# 7. Set up the pricing engine for the callable bond using the calibrated G2++ model
callable_bond_engine = ql.TreeCallableFixedRateBondEngine(model, 100)  # Using 100 time steps
callable_bond.setPricingEngine(callable_bond_engine)

# 8. Price the callable bond
callable_bond_price = callable_bond.cleanPrice()
print("Callable Bond Price: ", callable_bond_price)

Callable Bond Price:  99.01755207044552
